<a href="https://colab.research.google.com/github/catherinedoudou/BeautifulSoup-Waterstones/blob/main/cloud_shell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()    ##  get authentiaction from colab

In [2]:
!pip install requests_html > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [42]:
# from to_cloud import upload_to_bigquery, upload_to_gcs
import os
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from datetime import datetime
from google.cloud import bigquery
from google.cloud import storage
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO
import io
from PIL import Image
import time


def upload_to_bigquery(row_to_insert, table_id):
    # Instantiates a client
    bigquery_client = bigquery.Client(project='catherine-test-347809')

    # Prepares a reference to the dataset
    dataset_ref = bigquery_client.dataset('company')

    # Gets the table 
    table_ref = dataset_ref.table(table_id)
    table = bigquery_client.get_table(table_ref)  # API call

    errors = bigquery_client.insert_rows(table, row_to_insert)  # API request
    assert errors == []


def upload_to_gcs(bucket_name, folder_name, file_name): # Retrieve an existing bucket

  storage_client = storage.Client()
  bucket = storage_client.get_bucket(bucket_name)

  new_blob = bucket.blob(f'{folder_name}/{file_name}')  # folder_name/logo_name
  new_blob.upload_from_filename(f'/content/{file_name}')     # file_name looks like 'logo.png'
  
  if os.path.exists(file_name):     # delete the local file immediately after upload images into gcs succeed
    os.remove(file_name)
  else:
    print("The file does not exist")


def get_soup(url):
    cert = "zyte-smartproxy-ca.crt"

    ### Proxy settings
    proxy_host = "proxy.crawlera.com"
    proxy_port = "8011"
    proxy_auth = "f6b775febdec4c9b951e9097883fb411:"
    proxies = {
        "https": f"http://{proxy_auth}@{proxy_host}:{proxy_port}/",
        "http": f"http://{proxy_auth}@{proxy_host}:{proxy_port}/",
    }

    headers = {"X-Crawlera-Profile": "desktop"}

    page = HTMLSession().get(
        url, headers=headers, verify=True, allow_redirects=True, timeout=8.0)

    if page.ok:
        return BeautifulSoup(page.content, "html.parser")
    else:
        page = HTMLSession().get(
            url,
            headers=headers,
            proxies=proxies,
            verify=cert,
            allow_redirects=True,
            timeout=8.0,
        )
        return BeautifulSoup(page.content, "html.parser")


def get_image_lists(url):
    """
    This function returns all images but sort based on likelyhood to be a logo.
    """

    soup = get_soup(url)
    body = soup.find("body")
    header_images = []
    body_images = []
    domain = urlparse(url).netloc


    if "www." in domain:
        domain = domain.replace("www.", "")
    domain =urlparse(domain).path.split(".")[0]


    ## For every website, get all images,
    # but sort based on likelyhood to be a logo
    # (i.e. header_images > body_images (with the more logo likely to be at the start of body_images vs the end))
    src_attrs = ["src", "data-src"]

    if body and body.find("img"):
        add_to_end = []

        for i in body.find_all("img"):
            # Get body>header images (with 'logo' in)
            if i.findParents("header"):
                for src in src_attrs:
                  if i.get(src) and "logo" in i.get(src).lower():
                      header_images.append(i.get(src))

            # Scrape all body images (with 'logo' in, not inc. body>header images)
            elif not i.findParents("header"):
                for src in src_attrs:
                    if i.get(src) and "logo" in str(i).lower():
                        body_images.append(i.get(src))
                    elif i.get(src) and domain in str(i).lower():
                        body_images.append(i.get(src))

            # Scrape all other body images
            for src in src_attrs:
                if i.get(src) and i.get(src) not in header_images + body_images:
                    add_to_end.append(i.get(src))
        body_images += add_to_end

    return (header_images, body_images)  # return a tuple contains lists of image urls


def filter_images(url, image_url):
  if (
      image_url is not None
      and image_url.startswith("http") == False
      and image_url.startswith("/") == False  ### example : img/logo@3x.png
  ):
      image_url = url + "/" + image_url

  if image_url is not None and image_url.startswith("//"):  ### //img/logo@3x.png
      image_url = image_url.replace("//", "/")

  if (
      image_url is not None
      and image_url.startswith("http") == False
      and image_url.startswith("/")
  ):  ### example :/img/logo@3x.png
      image_url = url + image_url

  return image_url


def gcs_upload(image_url, domain, file_name, gcs_root, bucket_name):

    if image_url.endswith("svg"):
      filename = f"{file_name}.svg"  ## save image in its own format
    else:
      filename = f"{file_name}.png"

    if image_url is not None and HTMLSession().get(image_url).status_code == 200:    #image is not None and 
      
      byteImgIO = io.BytesIO()
      print(time.time())
      byteImg = Image.open(HTMLSession().get(image_url, timeout=3).content)
      byteImg.save(byteImgIO, filename)
      print(time.time())
      upload_to_gcs(bucket_name, domain, filename)
      gcs_logo_url = gcs_root + bucket_name + "/" + domain + "/" + filename
      return gcs_logo_url

      # byteImgIO.seek(0)
      # byteImg = byteImgIO.read()
      
     
      


def logo_url(url):
    """
    This function only gets the most possible logo url
    """
    header_images, body_images = get_image_lists(url)
    gcs_root = "https://storage.cloud.google.com/"
    bucket_name = "company_logo_bucket"
    domain = urlparse(url).netloc

    if "www." in domain:
        domain = domain.replace("www.", "")

    if header_images is not None and header_images != []:
        image_url = filter_images(url, header_images[0])
        gcs_logo_url = gcs_upload(image_url, domain, domain, gcs_root, bucket_name)
        for indx, image in enumerate(header_images[1:]):
           image_url = filter_images(url, image)
           try:
            gcs_upload(image_url, domain, indx, gcs_root, bucket_name)
           except:
              pass

        return gcs_logo_url  
      
    elif body_images is not None and body_images != []:
        image_url = filter_images(url, body_images[0])
        gcs_logo_url = gcs_upload(image_url, domain, gcs_root, bucket_name)
        for indx, image in enumerate(body_images[1:]):
           image_url = filter_images(url, image)
           try:
            gcs_upload(image_url, domain, indx, gcs_root, bucket_name)
           except:
              pass
              
        return gcs_logo_url
    
    


def bq_upload_logo(url):

    # credentials_path = "/app/config/servicekey.json"
    # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

  
    domain = urlparse(url).netloc

    if "www." in domain:
        domain = domain.replace("www.", "")

    client = bigquery.Client(project="asteroid0")
    QUERY = f"""SELECT CompanyName,CompanyNumber 
                FROM `asteroid0.Ceres.ukcompanies_elastic_indext` 
                WHERE URL LIKE '{domain}'
            """

    query_job = client.query(QUERY)

    rows = query_job.result()
    gcs_logo_url = logo_url(url)

    for row in rows:
        name = row["CompanyName"]
        company_num = row["CompanyNumber"]
        upload_to_bigquery(
            [
                {
                    "company_name": name,
                    "company_url": url,
                    "company_number": company_num,
                    "type": "logo",
                    "gcs_url": gcs_logo_url,
                    "current_timestamp": datetime.now(),
                }
            ],
            table_id="CompanyLogo",
            )
    return  {"comapny": domain, "gcs_logo_url": gcs_logo_url}

In [10]:
# def multithread(urls):
#     # start the thread pool
#     result = []
#     with ThreadPoolExecutor(10) as executor:
#         # execute tasks concurrently and process results in order
#         result = executor.map(bq_upload_logo, urls)
#     return result

In [30]:
from pandas.io import gbq 
# s = time.time()
client = bigquery.Client(project="asteroid0")
QUERY = f"""SELECT URL
            FROM `asteroid0.Ceres.ukcompanies_elastic_index`
            LIMIT 100 """

query_job = client.query(QUERY)
# d = time.time()
print(d-s)
df = gbq.read_gbq(QUERY, 'asteroid0')
urls = df['URL'].to_list()


0.7278323173522949


In [9]:
urls

['http://www.whitstableoystercompany.com',
 'http://www.gehealthcare.com',
 'http://www.swire.com',
 'http://www.thomasarmstrong.co.uk',
 'http://www.ashbrooks.co.uk',
 'http://www.fwhipkin.co.uk',
 'http://www.newby-trust.org.uk',
 'http://www.guycarpenter.com',
 'http://www.west-leigh.co.uk',
 'http://www.haldane-fisher.com',
 'http://www.williampowell.co.uk',
 'http://www.wescol.com',
 'http://www.suttonsgroup.com',
 'http://www.tamo.co.uk',
 'http://hillfoot.com',
 'http://www.siemensfinancialservices.co.uk',
 'http://www.tafishbuilders.co.uk',
 'http://www.crescent-theatre.co.uk',
 'http://www.pcproperties.co.uk',
 'http://www.bartlettgroup.com',
 'http://www.apg.net',
 'http://www.edbroking.com',
 'http://www.chargeurs-interlining.com',
 'http://www.shj.co.uk',
 'http://www.wolds-gliding.org',
 'http://www.hoaresbank.co.uk',
 'http://www.bita.org.uk',
 'http://www.carbank.co.uk',
 'http://www.greenscape-grounds-maintenance.co.uk',
 'http://www.hubacontrol.com',
 'http://www.polic

In [45]:
print(bq_upload_logo('http://www.marellimotori.com'))


1653383739.428741
{'comapny': 'marellimotori.com', 'gcs_logo_url': None}


In [113]:
for url in urls[:20]:
  print(bq_upload_logo(url))

{'comapny': 'whitstableoystercompany.com', 'gcs_logo_url': None}
{'comapny': 'gehealthcare.com', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/gehealthcare.com/gehealthcare.com.png'}
{'comapny': 'swire.com', 'gcs_logo_url': None}
{'comapny': 'thomasarmstrong.co.uk', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/thomasarmstrong.co.uk/thomasarmstrong.co.uk.png'}
{'comapny': 'ashbrooks.co.uk', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/ashbrooks.co.uk/ashbrooks.co.uk.png'}
{'comapny': 'fwhipkin.co.uk', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/fwhipkin.co.uk/fwhipkin.co.uk.png'}
{'comapny': 'newby-trust.org.uk', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/newby-trust.org.uk/newby-trust.org.uk.png'}
{'comapny': 'guycarpenter.com', 'gcs_logo_url': None}
{'comapny': 'west-leigh.co.uk', 'gcs_logo_url': 'https://storage.cloud.google.com/company_logo_bucket/west-leigh

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


{'comapny': 'bartlettgroup.com', 'gcs_logo_url': None}
